In [ ]:
import pyrankvote
from pyrankvote import Candidate, Ballot
import csv
import time

def run_election(votes_path, vote_columns_range, skip_rows_range, type="rcv", number_of_seats=2, ignored_candidates=[], candidate_aliases={}):
    candidates = {}
    ballots = []
    
    with open(votes_path, newline='') as csvfile:
        responses = csv.reader(csvfile, delimiter=',', quotechar='"')
        
        for x in skip_rows_range:
            next(responses);
            
        for row in responses:
            candidates_to_add = []
            for i in vote_columns_range:
                candidate_to_add = row[i].strip()
                
                if candidate_to_add: #if vote is not empty
                    if candidate_to_add in candidate_aliases: #if there exists an alias for this candidate
                        candidate_to_add = candidate_aliases[candidate_to_add] #set the candidate name as per the alias
                    if candidate_to_add not in ignored_candidates: #if this candidate is not ignored
                        if candidate_to_add not in candidates: #if there does not yet exist a Candidate object for this candidate
                            candidates[candidate_to_add] = Candidate(candidate_to_add) #create a Candidate object for this candidate
                        if candidates[candidate_to_add] not in candidates_to_add: #if this voter has not already chosen this candidate for a higher-priority choice
                            candidates_to_add.append(candidates[candidate_to_add]) #add this candidate to the list of candidates for this ballot
            if candidates_to_add: #if not all choices are empty
                #print(candidates_to_add)
                ballots.append(Ballot(ranked_candidates=candidates_to_add)) #add this voter's ballot to the list of ballots
                
        #print(candidates)
        
        candidates_list = []
        for i in candidates:
            candidates_list.append(candidates[i])
        if type=="rcv":
            result = pyrankvote.instant_runoff_voting(candidates_list, ballots)
        elif type=="stv":
            result = pyrankvote.single_transferable_vote(candidates_list, ballots, number_of_seats)
        print(result)


path = input("Enter name of responses CSV (ensure it is located in the current directory):\n")
with open(path, newline='') as csvfile:
    responses = csv.reader(csvfile, delimiter=',', quotechar='"')
while(True):
    list_of_positions = input("Provide a comma-seperated list of the names of the positions, in the order that they occur in the responses CSV:\n")
    list_of_positions = [i.strip() for i in list_of_positions.split(',')]
    print("You entered",len(list_of_positions),"positions. Does this look right?\n", list_of_positions)
    if input("y/n: ") == 'y':
        break
max_choices = int(input("What is the maximum number of choices a voter is allowed per position? (typically 3):\n"))
skipped_rows = int(input("Enter the number of rows to skip at the top of the CSV (typically at least 1, so that the row with the column names is skipped):\n"))
skipped_columns = int(input("Enter the number of columns to skip at the left of the CSV (typically at least 1, so the timestamp column is skipped, but usually more so that the dues verification responses are also skipped):\n"))

print("\nStarting election...")
time.sleep(1)

while(True):    
    current_position = input("Enter the name of the position for which you wish to run an election:\n")
    election_counter = list_of_positions.index(current_position)
    while(True):
        print("\nNow running election for " + current_position)
        
        if input("Is this a multi-seat position (e.g. Co-Chair)? y/n: ") == 'y':
            election_type = "stv"
            number_of_positions = int(input("How many candidates can be elected to this position?: "))
        else:
            election_type = "rcv"
            number_of_positions = 1;
            
        if input("Would you like to ignore any candidates in this election? y/n: ") == 'y':
            ignored = input("Provide a comma-seperated list of the names of the candidates you wish to ignore:\n")
            ignored = [i.strip() for i in ignored.split(',')]
        else:
            ignored = []
            
        if input("Would you like to provide any aliases for candidates (typically used for inconsistently spelled/capitalized write-in candidates)? y/n: ") == 'y':
            aliases = {}
            while(True):
                incorrect = input("Enter a name as it is spelled on one or more ballots:\n")
                correct = input("Enter the name of the candidate you wish to alias the previous name to:\n")
                aliases[incorrect] = correct
                if input("Would you like to enter more aliases? y/n: ") != 'y':
                    break
        else:
            aliases = {}
            
        print();
        print("Ignored candidates:", ignored)
        print("Candidate aliases:", aliases)
        run_election(path, range(skipped_columns+election_counter*max_choices,skipped_columns+election_counter*max_choices+max_choices), range(0,skipped_rows), type=election_type, number_of_seats=number_of_positions, ignored_candidates=ignored, candidate_aliases=aliases)
        
        if input("Proceed to next election? y/n: ") == "y":
            break;
    

Enter name of responses CSV (ensure it is located in the current directory):
 UCSB YDSA 2024 Elections (Responses) - Form Responses 1.csv
Provide a comma-seperated list of the names of the positions, in the order that they occur in the responses CSV:
 Outreach Chair, Political Education Chair, Internal Affairs Chair, Co-Chair, Treasurer, Secretary, Labor Chair


You entered 7 positions. Does this look right?
 ['Outreach Chair', 'Political Education Chair', 'Internal Affairs Chair', 'Co-Chair', 'Treasurer', 'Secretary', 'Labor Chair']


y/n:  y
What is the maximum number of choices a voter is allowed per position? (typically 3):
 3
Enter the number of rows to skip at the top of the CSV (typically at least 1, so that the row with the column names is skipped):
 1
Enter the number of columns to skip at the left of the CSV (typically at least 1, so the timestamp column is skipped, but usually more so that the dues verification responses are also skipped):
 3



Starting election...


Enter the name of the position for which you wish to run an election:
 Outreach Chair


...


Ignored candidates: []
Candidate aliases: {}
FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Sofia Casano        29  Elected
Zac Sherar           2  Rejected
Nate Behrendt        0  Rejected



Proceed to next election? y/n:  y
Enter the name of the position for which you wish to run an election:
 Labor Chair



Now running election for Labor Chair


Is this a multi-seat position (e.g. Co-Chair)? y/n:  n
Would you like to ignore any candidates in this election? y/n:  n
Would you like to provide any aliases for candidates (typically used for inconsistently spelled/capitalized write-in candidates)? y/n:  n



Ignored candidates: []
Candidate aliases: {}
FINAL RESULT
Candidate           Votes  Status
----------------  -------  --------
Clay Dau               24  Elected
Michael Collin          5  Rejected
Isabelle Prittie        2  Rejected

